In [10]:
#! bash install.sh
%run prelude.py

# Aggregating Percentiles -- A case study

Calculating accurate percentiles over aggergated datasets is necessary for basic analysis of API performance:

- Aggregation across time
- Aggregation across different nodes
- Aggregation across different endpoints

# So I said "Percentiles can't be aggregated" ...

* Blog: www.circonus.com/problem-math/

* Twitter: https://twitter.com/heinrichhartman/status/748562001392111617

* Reddit: https://www.reddit.com/r/devops/comments/941n2k/tsdbs_at_scale_part_one/e3po8d3/

* Gil Tene - How not to measure latency -- https://www.youtube.com/watch?v=lJ8ydIuPFeU&feature=youtu.be&t=9m15s".

# Let's have a closer look ...

In [45]:
# API Latency Dataset
FILENAME="../datasets/api_latencies_24x1h.tsv"
XAL = defaultdict(list)
with open(FILENAME) as fh:
    for line in fh:
        # line = "3 232.123321" -- hour latency
        a, b = line.split("\t")
        XAL[int(a)].append(float(b))

In [43]:
P = [0, 50, 90, 99, 99.9, 100]

In [44]:
H("API Latencies over time")

PH = [ "p{}".format(p) for p in P]
print(tabulate([ [h, len(X)] + [ np.percentile(X,p) for p in P ] for h, X in XAL.items() ], headers=["hour", "count"] +  PH, floatfmt=".1f" ))

  hour    count    p0    p50    p90     p99    p99.9     p100
------  -------  ----  -----  -----  ------  -------  -------
     0    35212   0.3   12.0   19.8   113.0    448.0   1406.1
     1    46133   0.4   16.6   23.4   115.0    604.4   1548.5
     2    35794   0.4   20.6   27.5   117.1    925.4   3001.0
     3    42859   0.3   20.8   35.1   424.4   1652.6   2776.4
     4    18253   0.4   23.2   42.7   497.0   1232.0   2195.7
     5      599   0.4   24.3   70.7  2679.9   3215.7   3298.5
     6      280   0.4   25.5   95.4   153.6    182.3    191.1
     7       68   7.2   26.9   87.0   111.0    124.0    125.5
     8       99   0.4   24.9   70.7   119.4    135.5    137.3
     9       80   7.7   33.7  165.8  1648.3   4747.1   5091.4
    10       25   7.6   25.4   59.0    81.4     83.0     83.2
    11       49   0.6   26.3   77.3    93.4     95.6     95.9
    12      105   7.5   25.2   75.8   115.0    235.3    249.3
    13      112   5.1   25.2   76.1   115.4    117.0    117.1
    14  

## (0) Aggregating the Dataset itself

In [39]:
XT = []
for X in XAL.values():
    for x in X:
        XT.append(x)
        
TOTP = [ np.percentile(XT, p) for p in P ]

TAB([["total"] + TOTP], headers=[""] + PH)

             p0      p50      p90      p99    p99.9     p100
-----  --------  -------  -------  -------  -------  -------
total  0.293543  20.9436  35.7746  1114.71  5991.83  10990.4


## (1) Averaging Percentiles

In [40]:
AVGP = [ np.mean([  np.percentile(X,p) for X in XAL.values() ]) for p in P  ]
HEAD=" p     p-total  p-avg    err                   err-%".split()
TAB([[ P[i], TOTP[i], AVGP[i], abs(TOTP[i]-AVGP[i]), abs(TOTP[i]-AVGP[i])/TOTP[i] * 100 ] for i in range(len(P)) ], floatfmt=".1f", headers=HEAD)

    p    p-total    p-avg     err    err-%
-----  ---------  -------  ------  -------
  0.0        0.3      1.9     1.6    540.9
 50.0       20.9     22.3     1.3      6.4
 90.0       35.8     60.3    24.5     68.5
 99.0     1114.7    719.4   395.3     35.5
 99.9     5991.8   1635.0  4356.8     72.7
100.0    10990.4   2664.3  8326.1     75.8


### Discussion

- Errors of >50% are common

## (2) Weighted Averaged Percentiles

Idea: Let's take into account the number of samples collected each hour

In [41]:
WAVGP = [ np.sum([ len(X) * np.percentile(X,p) for X in XAL.values() ]) / np.sum( [ len(X) for X in XAL.values() ]) for p in P  ]
# TAB([ ["w-avg-percentiles"] + WAVGP ], headers=[""] + P)
HEAD="p      p-total  p-wavg   err                   err-%".split()
TAB([[ P[i], TOTP[i], WAVGP[i], abs(TOTP[i]-WAVGP[i]), abs(TOTP[i]-WAVGP[i])/TOTP[i] * 100 ] for i in range(len(P)) ], floatfmt=".1f", headers=HEAD)

    p    p-total    p-wavg     err    err-%
-----  ---------  --------  ------  -------
  0.0        0.3       0.3     0.0     16.4
 50.0       20.9      20.8     0.1      0.5
 90.0       35.8      39.7     3.9     10.9
 99.0     1114.7    1306.6   191.9     17.2
 99.9     5991.8    3197.5  2794.4     46.6
100.0    10990.4    5661.6  5328.8     48.5


### Discussion

- Errors are kept <50%. Still not very useful.

# Discussion

* Averaging Percentiles runs into essentially unbounded errors

* Weighting by count(), does somewhat help, but does not fix the problem

* Contrast this to the situation with the count-below metrics, which are perfectly mergable and carry similar information

# Mergable Percentile Summaries

There are multiple methods available to summarize data in a mergable way, that allows for percentile calculations.

1. Circllhists -- Schlossnagle @ Circonus 2013

2. HDR Histograms -- Tene @ Azul Systems, 2015

3. t-digest -- Dunning @ MapR, Erl @ Dynatrace

...

4. DD-sketch -- ? @ DataDog 2019
